In [ ]:
import os
import time
import random
import sys

suits = ['♠', '♥', '♦', '♣']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
deck = [rank + suit for suit in suits for rank in ranks]

class Board:
    def __init__(self):
        self.num_players = self.get_players()
        self.player_hands, self.cards = self.deal(self.num_players)

    def get_players(self):
        players = 0
        while players < 2 or players > 5:
            try:
                players = int(input("How many players are playing? "))
                if players < 2 or players > 5:
                    print("The number of players must be between 2 and 5!")
            except ValueError:
                print("Please enter a number.")
        print(f"There are {players} players.")
        self.clear_screen()
        return players

    def clear_screen(self):
   
        time.sleep(0.75)
        print("\nSCREEN CLEARING...")
        time.sleep(1.75)
        try:
            from IPython.display import clear_output
            clear_output(wait=True)
        except ImportError:
            pass
        if os.name == 'nt':
            os.system('cls')
        else:
            os.system('clear')

    def deal(self, num_players):
        cards = deck.copy()
        random.shuffle(cards)
        hands = []
        for _ in range(num_players):
            hand = [cards.pop(), cards.pop()]
            hands.append(hand)
        return hands, cards

    def hand_value(self, hand):
        value = 0
        aces = 0
        for card in hand:
            rank = card[:-1]
            if rank in ['J', 'Q', 'K']:
                value += 10
            elif rank == 'A':
                value += 11
                aces += 1
            else:
                value += int(rank)
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def play_round(self):
        for i, hand in enumerate(self.player_hands, 1):
            print(f"\nPlayer {i}'s turn!")
            while True:
                print(f"Your hand: {hand} (value: {self.hand_value(hand)})")
                action = input("Do you want to 'hit' or 'stand'? ").lower()
                if action == 'hit':
                    hand.append(self.cards.pop())
                    if self.hand_value(hand) > 21:
                        print(f"Bust! Your hand: {hand} (value: {self.hand_value(hand)})")
                        self.clear_screen()
                        break
                elif action == 'stand':
                    self.clear_screen()
                    break
                else:
                    print("Invalid input, please type 'hit' or 'stand'.")
                if len(hand) == 5 and self.hand_value(hand) <= 21:
                    self.clear_screen()
                    print(f"Player {i} wins by the 5 card rule!")
                    sys.exit()

    def check_winner(self):
        still_in = []
        for i, hand in enumerate(self.player_hands, 1):
            value = self.hand_value(hand)
            if value <= 21:
                still_in.append((i, value))

        if not still_in:
            print("All players busted. No winner!")
        else:
            max_score = max(score for _, score in still_in)
            winners = [player for player, score in still_in if score == max_score]

            if len(winners) == 1:
                print(f"\nPlayer {winners[0]} wins with a score of {max_score}!")
            else:
                print("There was a tie!")

game = Board()
game.play_round()
game.check_winner()

TypeError: Board.play_round() missing 1 required positional argument: 'self'